In [ ]:
import json
from tqdm import tqdm
import spacy
import medspacy
from medspacy.visualization import visualize_ent
from cxr_target_rules import rules

In [16]:
nlp = medspacy.load()

In [ ]:
target_matcher = nlp.get_pipe("medspacy_target_matcher")

target_rules = rules

target_matcher.add(target_rules) # type: ignore

In [18]:
def to_bio_format(doc):
    lines = []
    for sent in doc.sents:
        for token in sent:
            label = "O"
            for ent in doc.ents:
                if token.idx >= ent.start_char and token.idx < ent.end_char:
                    prefix = "B" if token.idx == ent.start_char else "I"
                    label = f"{prefix}-{ent.label_}"
                    break
            lines.append(f"{token.text} {label}")
        lines.append("")  # blank line between sentences
    return lines

In [19]:
def process_and_save(jsonl_path, output_path):
    with open(jsonl_path, 'r') as infile:
        texts = [json.loads(line)["text"] for line in infile if "text" in line]
    bio_dataset = []
    for doc in nlp.pipe(texts, batch_size=32):
        bio_dataset.extend(to_bio_format(doc))
            
    with open(output_path, 'w') as outfile:   
        for line in bio_dataset:
                    outfile.write(line + "\n")
    
    print("Done!")
                # ents = [
                #             {
                #                 "id": id,
                #                 "text": ent.text,
                #                 "label": ent.label_,
                #                 "is_negated": ent._.is_negated,
                #                 "section": ent._.section_category,
                #             }
                #             for ent in doc.ents
                # ]
                # for ent_json in ents:
                #             outfile.write(json.dumps(ent_json) + "\n")

In [21]:
input_dir = "./mimic_cxr_train_val.jsonl" 
output_bio_file = "bio_dataset_1000.txt"
process_and_save(input_dir, output_bio_file)

Done!
